In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, State

In [2]:
app = Dash(__name__)

df = pd.read_csv('nj_teachers_salaries_2016.csv')
df.drop(columns=['last_name', 'first_name', 'fte'], inplace=True)
df.dropna(inplace=True)
df = df.loc[(df!='X').all(axis=1)]

# -------------------- Select categorical feature -----------------------------
select_category = dcc.Dropdown(options=[{'label': 'Certificate', 'value': 'certificate'},
                                     {'label': 'Subcategory', 'value': 'subcategory'},
                                     {'label': 'Teaching route', 'value': 'teaching_route'},
                                     {'label': 'Hightly qualified', 'value': 'highly_qualified'}],
                            value=None, 
                            placeholder='Select',
                            id='select-category',
                            style={'font-size': '14px', 'flex': '1'})

category_container = html.Div(children=['Category: ',
                                        select_category],
                              id='category-container',
                              style={
                                  'display': 'flex',
                                  'font-size': '12px',
                                  'align-items': 'center',
                                  'width': '100%'
                              })

# --------------------- Select county ------------------------------------------------
county = df['county'].unique()
select_county = dcc.Dropdown(options=county,
                             value=county[0],
                             clearable=False,
                             id='select-county',
                             style={'flex': '1', 'font-size': '16px'})

county_container = html.Div(children=['County: ', select_county],
                            id='county-container',
                            style={
                                'display': 'flex', 
                                'font-size': '12px', 
                                'align-items': 'center'
                            })

# ------------------------ Select district -----------------------------------------
check_type = True
select_check_type = dcc.Dropdown(options=[{'label': 'select one', 'value': True}, 
                                          {'label': 'select many', 'value': False}],
                                value=check_type,
                                clearable=False,
                                id='select-check-type',
                                style={'flex': '1'})

check_all = dcc.Checklist(options=['All'],
                          value=[],
                          id='check-all',
                          style={})

check_container = html.Div(children=['District: ',
                                     select_check_type,
                                     check_all], 
                           id='check-container',
                           style={
                               'font-size': '12px',
                               'display': 'flex',
                               'width': '160px',
                               'align-items': 'center'
                           })

def select_district(options, type_, id_, style, labelStyle):
    if type_:
        return dcc.RadioItems(options=options, 
                              value=[], 
                              id=id_, 
                              style=style, 
                              labelStyle=labelStyle)
    
    return dcc.Checklist(options=options, 
                         value=[], 
                         id=id_, 
                         style=style, 
                         labelStyle=labelStyle)

options = df.loc[df['county']==county[0], 'district'].unique()
id_ = 'select-district'

select_district_style = {
    'display': 'flex',
    'flex-direction': 'column',
    'width': '160px',
    'height': '500px',
    'overflow-y': 'scroll',
    'margin-right': '0'
}

select_district_labelStyle = {
    'width': '100%',
    'font-size': '14px',
    'padding': '4px 0'
}

district_container = html.Div(children=[check_container,
                                        select_district(options=options, 
                                                        type_=check_type,
                                                        id_=id_,
                                                        style=select_district_style,
                                                        labelStyle=select_district_labelStyle)],
                              id='district-container',
                              style={'margin-top': '6px'})

# ------------------------------ Select graph theme -------------------------------------------
select_graph_theme = dcc.RadioItems(options=[{'label': 'light', 'value': 'seaborn'}, 
                                             {'label': 'dark', 'value': 'plotly_dark'}],
                                  value='seaborn',
                                  id='select-graph-theme',
                                  style={'flex': '1', 'font-size': '14px'})

graph_theme_container = html.Div(children=['Mode: ', select_graph_theme],
                                 id='graph-theme-container',
                                 style={
                                        'margin-top': '6px',
                                        'display': 'flex', 
                                        'align-items': 'center',
                                        'font-size': '12px',
                                        'width': '160px'})


# ------------------------------ Filter container ---------------------------------------------
filters_container = html.Div(children=[category_container,
                                       county_container,
                                       district_container,
                                       graph_theme_container], 
                             style={
                                 'position': 'fixed',
                                 'top': '0',
                                 'left': '0',
                                 'width': '160px'
                             })

# ---------------------------------- Graph -----------------------------------------------------
graph = dcc.Graph(id='graph', style={'height': '100%'})
graph_title = html.H2(children=['Teacher\'s salary in New Jersey in 2016'], style={'margin': '0'})

graph_container = html.Div(children=[graph_title,
                                     graph],
                           id='graph-container',
                           style={
                               'margin-left': '154px', 
                               'width': '100%', 
                               'height': '1000px'})

# Define layout of Dashboard -------------------------------------------------------------------
app.layout = html.Div(children=[html.Div(children=[filters_container]),
                                 graph_container],
                      style={'display': 'flex'})

# processing fitler ------------------------------------------------
@app.callback(Output('select-district', 'options'),
              Input('select-county', 'value'))
def update_options(value):
    return df.loc[df['county']==value, 'district'].unique()

@app.callback(Output('check-all', 'options'), 
              Output('check-all', 'value'), 
              Input('select-check-type', 'value'))
def disable_all_check(value):
    return [{'label': 'All', 'value': 'All', 'disabled': value}], []

@app.callback(Output('district-container', 'children'), 
              Input('select-check-type', 'value'),
              Input('select-district', 'options'),
              State('select-district', 'id'),
              State('select-district', 'style'),
              State('select-district', 'labelStyle'),
              State('district-container', 'children'))
def change_select_district(value, options, id_, style, labelStyle, children):
    children[-1] = select_district(options=options, 
                                  type_=value, 
                                  id_=id_, 
                                  style=style, 
                                  labelStyle=labelStyle)
    return children

@app.callback(Output('select-district', 'value'),
              Input('check-all', 'value'),
              State('select-district', 'options'))
def get_check_values(value, options):
    return options if value else []

# processing graph --------------------------------------------------
@app.callback(Output('graph', 'figure'), 
              Input('select-district', 'value'),
              Input('select-graph-theme', 'value'),
              Input('select-category', 'value'))
def update_graph(districts, template, category):
    if type(districts) != type([]):
        districts = [districts]
    dff = df.loc[df['district'].isin(districts)]
    check = dff.values.tolist() != []

    fig = make_subplots(rows=3, cols=3, specs=[[{'rowspan': 2, 'colspan': 2}, None, {}], 
                                               [None, None, {}], 
                                               [{'type': 'pie'}, {'colspan': 2}, None]])
    
    fig.add_traces(data=px.scatter(data_frame=dff, 
                                   x='experience_total', 
                                   y='salary', 
                                   color=category, 
                                   trendline='ols').update_traces(showlegend=False).data if check else None,
                   rows=1, cols=1)
    
    fig['layout']['xaxis'].update(title='experience_total')
    fig['layout']['yaxis'].update(title='salary')
    
    fig.add_traces(
        data=px.histogram(data_frame=dff, 
                          x='salary', 
                          color=category, 
                          barmode='overlay').update_traces(bingroup='a', showlegend=False).data if check else None,
        rows=1, cols=3)
    fig['layout']['xaxis2'].update(title='salary')
    
    fig.add_traces(
        data=px.histogram(data_frame=dff.copy(), 
                          x='experience_total', 
                          color=category, 
                          barmode='overlay').update_traces(bingroup='b', showlegend=False).data if check else None,
        rows=2, cols=3)
    fig['layout']['xaxis3'].update(title='experience_total')
    
    
    if category:
        fig.add_traces(data=px.pie(data_frame=dff,
                                   names=category,
                                   hole=0.2).update_traces(showlegend=False).data if check else None, 
                       rows=3, cols=1)
    
    fig.add_traces(
        data=px.box(data_frame=dff, 
                    x=category, 
                    y='salary', 
                    color=category).data if check else None,
        rows=3, cols=2)
    fig['layout']['xaxis4'].update(title=category, showticklabels=False)
    fig['layout']['yaxis4'].update(title='salary')
        
    legend={
        'title': category,
        'xanchor': 'left',
        'x': 0,
        'yanchor': 'bottom',
        'y': 1,
        'orientation': 'h'
    }
    fig.update_layout(template=template, legend=legend)
    return fig


# ------------------------------- RUN APP --------------------------------------------
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 13:20:17] "POST /_dash-update-component HTTP/1.